In [ ]:
import pandas as pd
import mlflow
from data_preprocessing import read_from_db
from data_preprocessing import extract_features, encode_categorical
import hydra

In [ ]:
@hydra.main(config_path='conf', config_name='config.yaml')
def predict(config, model, X, artifacts):
    def prepare_for_inference(df: pd.DataFrame) -> pd.DataFrame:
        ohe, schema, most_recent_vals = artifacts
        df = extract_features(df, inference=True)
        df = df.rename(columns={'value': 'lag'})
        df, _ = encode_categorical(df, ohe=ohe, fit=False)
        df[['rolling_mean', 'rolling_std']] = most_recent_vals
        X_recent = df[schema]
        return X_recent

    data_inference = read_from_db(config.data.conn_params)
    df_inference = data_inference.tail(1)
    
    X_recent = prepare_for_inference(df_inference, artifacts)
    
    y_pred = model.predict(X_recent)
    return y_pred

In [ ]:
if __name__ == "__main__":
    predict()